In [32]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [34]:
data = pd.read_csv('train_1.csv').fillna(0)

In [35]:
data

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145058,Underworld_(serie_de_películas)_es.wikipedia.o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,12.0,13.0,3.0,5.0,10.0
145059,Resident_Evil:_Capítulo_Final_es.wikipedia.org...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145060,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145061,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
data = pd.melt(data[list(data.columns)], id_vars= 'Page', var_name= 'date', value_name= 'visit')
data

,Page,date,visit
0,2NE1_zh.wikipedia.org_all-access_spider,2015-07-01,18.0
1,2PM_zh.wikipedia.org_all-access_spider,2015-07-01,11.0
2,3C_zh.wikipedia.org_all-access_spider,2015-07-01,1.0
3,4minute_zh.wikipedia.org_all-access_spider,2015-07-01,35.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2015-07-01,0.0
...,...,...,...
79784645,Underworld_(serie_de_películas)_es.wikipedia.o...,2016-12-31,10.0
79784646,Resident_Evil:_Capítulo_Final_es.wikipedia.org...,2016-12-31,0.0
79784647,Enamorándome_de_Ramón_es.wikipedia.org_all-acc...,2016-12-31,0.0
79784648,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,2016-12-31,0.0


In [37]:
data['date'] = data['date'].astype('datetime64[ns]')
data['date']

0          2015-07-01
1          2015-07-01
2          2015-07-01
3          2015-07-01
4          2015-07-01
              ...    
79784645   2016-12-31
79784646   2016-12-31
79784647   2016-12-31
79784648   2016-12-31
79784649   2016-12-31
Name: date, Length: 79784650, dtype: datetime64[ns]

In [38]:
# This gets us a list of all unique page name
un = data['Page'].unique()
un

array(['2NE1_zh.wikipedia.org_all-access_spider',
       '2PM_zh.wikipedia.org_all-access_spider',
       '3C_zh.wikipedia.org_all-access_spider', ...,
       'Enamorándome_de_Ramón_es.wikipedia.org_all-access_spider',
       'Hasta_el_último_hombre_es.wikipedia.org_all-access_spider',
       'Francisco_el_matemático_(serie_de_televisión_de_2017)_es.wikipedia.org_all-access_spider'],
      dtype=object)

In [39]:
#Random page name
un[3]

'4minute_zh.wikipedia.org_all-access_spider'

**Grouping data**

In [40]:
gp = data.groupby(['Page'])

In [41]:
gp

**Normalize Data**

In [42]:
ser = gp.get_group(un[3])['visit']
ser = np.array(ser)
ser = ser.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0,1))
Normalized = scaler.fit_transform(ser)
Normalized

array([[0.11258278],
       [0.0397351 ],
       [0.02980132],
       [0.30794702],
       [0.00993377],
       [0.08278146],
       [0.04304636],
       [0.02649007],
       [0.03311258],
       [0.04966887],
       [0.04966887],
       [0.03311258],
       [0.07284768],
       [0.47682119],
       [0.04304636],
       [0.05298013],
       [0.2781457 ],
       [0.00993377],
       [0.09602649],
       [0.06953642],
       [0.02649007],
       [0.02980132],
       [0.03311258],
       [0.01986755],
       [0.01986755],
       [0.03311258],
       [0.02649007],
       [0.03311258],
       [0.14238411],
       [0.02317881],
       [0.04304636],
       [0.05960265],
       [0.02980132],
       [0.05298013],
       [0.05298013],
       [0.02980132],
       [0.01986755],
       [0.02980132],
       [0.        ],
       [0.02317881],
       [0.08609272],
       [0.05960265],
       [0.04966887],
       [0.00331126],
       [0.27483444],
       [0.06953642],
       [0.04304636],
       [0.152

In [43]:
# x = gp.get_group(un[3]).iloc[0:550]['visit']

In [44]:
Normalized.shape

(550, 1)

In [45]:
features = Normalized

**Passing the Normalized data to a Time Series Generator to convert data into an array which could be used to train the model**

In [46]:
ts_generator = TimeseriesGenerator(features, features, length=3, batch_size =1)

In [47]:
ts_generator[0]

(array([[[0.11258278],
         [0.0397351 ],
         [0.02980132]]]),
 array([[0.30794702]]))

**Taking the input from the user and then Normalizing it**

In [48]:
seq = [5,10,20]
seq = np.array(seq)
seq = seq.reshape(-1,1)
Normalizedseq = scaler.fit_transform(seq)
Normalizedseq

array([[0.        ],
       [0.33333333],
       [1.        ]])

**LSTM MODEL**

In [49]:
model = Sequential()
model.add(LSTM(20, activation = 'relu', input_shape = (3, 1)))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [50]:
model.fit(ts_generator, steps_per_epoch=1, epochs=20, verbose=0)

In [51]:
Normalizedseq.shape

(3, 1)

**Model Prediction**

In [52]:
Normalizedseq = np.array(Normalizedseq).reshape((1, 3, 1))
finalop = model.predict(Normalizedseq)

**Converting the Normalized output to its original state**

In [53]:
finalop = scaler.inverse_transform(finalop)
finalop

array([[6.079018]], dtype=float32)